## Regular Albert



In [0]:
!git clone https://github.com/acmilannesta/Bert-embedding
!git clone https://github.com/acmilannesta/albert

Cloning into 'Bert-embedding'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 123 (delta 54), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (123/123), 11.49 MiB | 6.21 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Cloning into 'albert'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 31 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [0]:
!mkdir ./input
!mkdir ./output
!mkdir ./albert_base
!wget https://tfhub.dev/google/albert_base/2?tf-hub-format=compressed
!tar -xf 2?tf-hub-format=compressed --directory ./albert_base
!pip install -r albert/requirements.txt

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os, gc
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm
import tensorflow as tf
print(tf.__version__)
print('grpc://'+os.environ['COLAB_TPU_ADDR'])
# import logging
tf.get_logger().setLevel('DEBUG')

1.15.0
grpc://10.122.181.98:8470


In [0]:
train = pd.read_csv('Bert-embedding/CDC/train.csv')
test = pd.read_csv('Bert-embedding/CDC/test.csv')
# Event weight
wt = pd.DataFrame(train.event.value_counts()/len(train)).rename(columns={'event':'weight'})
wt['event'] = wt.index
train = train.merge(wt, how='left', on='event')
# Reassign eventcode
train['event_idx'] = train.event.map({y:x for x, y in enumerate(np.sort(train.event.unique()))})
# Assign weight frequency
train['wt_freq'] = np.where(train.weight < 0.01, 1, np.where(train.weight < 0.05, 2, 3))

In [0]:
def convert_data(data_df, branch='train'):
    data_df = data_df.reset_index(drop=True)
    data_df['id'] = data_df.index
    data_df['alpha'] = 'a'
    if branch=='test':
        data_df = data_df[['id', 'text']]
        data_df.to_csv('/content/input/{:}.tsv'.format(branch), sep='\t', index=False)
    else:
        data_df = data_df[['id', 'event_idx', 'alpha', 'text']]
        data_df.to_csv('/content/input/{:}.tsv'.format(branch), sep='\t', index=False, header=False)

In [0]:
BATCH_SIZE = 16
NUM_EPOCHS = 3

In [0]:
# Fold results: 0.8736726
# F1_score: 0.8634 / 
kf = StratifiedKFold(n_splits=5)
idx = [x for x in kf.split(train, train.wt_freq)]
test_pred = np.zeros((len(test), 48))
train_copy = train.copy()
for i, (tr_idx, val_idx) in enumerate(idx, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train_copy.loc[tr_idx], train_copy.loc[val_idx]
    convert_data(tr)
    convert_data(val, 'dev')
    if (len(tr) * NUM_EPOCHS) % BATCH_SIZE != 0:
       print(len(tr) * NUM_EPOCHS // BATCH_SIZE)
    else:
       print(len(tr) * NUM_EPOCHS // BATCH_SIZE - 1)

    # os.system(
    !python -m albert.run_classifier_sp \
        --data_dir=input \
        --task_name=cola \
        --albert_config_file=albert_base/assets/albert_config.json \
        --vocab_file=albert_base/assets/30k-clean.vocab \
        --spm_model_file=albert_base/assets/30k-clean.model \
        --init_checkpoint=albert_base/variables/variables \
        --do_train=True \
        --do_eval=True \
        --train_batch_size=16 \
        --eval_batch_size=16 \
        --max_seq_length=128 \
        --learning_rate=1e-4 \
        --num_train_epochs=3 \
        --save_checkpoints_steps=5000 \
        --output_dir=output \
        --use_tpu \
        --tpu_name=grpc://10.122.181.98:8470
        # ')

    convert_data(test, 'test')
    # os.system(
    !python -m albert.run_classifier_sp \
        --task_name=cola \
        --do_predict=True \
        --data_dir=input \
        --albert_config_file=albert_base/assets/albert_config.json \
        --vocab_file=albert_base/assets/30k-clean.vocab \
        --spm_model_file=albert_base/assets/30k-clean.model \
        --init_checkpoint=output/model.ckpt-23093 \
        --max_seq_length=128 \
        --predict_batch_size=8 \
        --output_dir=output
        # ')
    pred = pd.read_csv('/content/output/test_results.tsv', sep='\t', header=None)
    test_pred += np.array(pred) / kf.get_n_splits()
    np.save('drive/My Drive/CDC Model/oof/test_albert_oof.npy', test_pred)

    convert_data(val, 'test')
    # os.system(
    !python -m albert.run_classifier_sp \
        --task_name=cola \
        --do_predict=True \
        --data_dir=input \
        --albert_config_file=albert_base/assets/albert_config.json \
        --vocab_file=albert_base/assets/30k-clean.vocab \
        --spm_model_file=albert_base/assets/30k-clean.model \
        --init_checkpoint=output/model.ckpt-15395 \
        --max_seq_length=128 \
        --predict_batch_size=8 \
        --output_dir=output
        # ')
    pred1 = pd.read_csv('output/submit_results.tsv', sep='\t', header=None)
    train.loc[val_idx, 'oof_pred'] = pred1.loc[:, 1].values
    score = f1_score(val.event_idx, pred1.loc[:, 1], average='weighted')
    with open('drive/My Drive/CDC Model/albert_score.txt', 'a') as writer:
        print('Fold - {:} - f1_score - {:.4f}'.format(i, score), file=writer)
    train.to_csv('drive/My Drive/CDC Model/oof/train_albert_oof.csv', index=False)
    !rm -r output
    !mkdir output

In [0]:
pred = pd.read_csv('input/test.tsv', sep='\t', header=None)
print(pred.shape)

(30793, 2)


#Albert model modified

In [0]:
!git clone https://github.com/acmilannesta/Bert-embedding
!mkdir ./input
!mkdir ./output
# !mkdir ./albert_base
!git clone https://github.com/acmilannesta/ALBERT-TF2.0
!pip install -r /content/ALBERT-TF2.0/requirements.txt

In [0]:
import os, gc
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm
import tensorflow as tf
print(tf.__version__)
import logging
tf.get_logger().setLevel(logging.ERROR)

2.0.0


In [0]:
train = pd.read_csv('Bert-embedding/CDC/train.csv')
test = pd.read_csv('Bert-embedding/CDC/test.csv')
# Event weight
wt = pd.DataFrame(train.event.value_counts()/len(train)).rename(columns={'event':'weight'})
wt['event'] = wt.index
train = train.merge(wt, how='left', on='event')
# Reassign eventcode
train['event_idx'] = train.event.map({y:x for x, y in enumerate(np.sort(train.event.unique()))})
# Assign weight frequency
train['wt_freq'] = np.where(train.weight < 0.01, 1, np.where(train.weight < 0.05, 2, 3))

In [0]:
def convert_data(data_df, branch='train'):
    data_df = data_df.reset_index(drop=True)
    data_df['id'] = data_df.index
    data_df['alpha'] = 'a'
    if branch=='test':
        data_df = data_df[['id', 'text']]
    else:
        data_df = data_df[['id', 'event_idx', 'alpha', 'text']]
    data_df.to_csv('/content/input/{:}.tsv'.format(branch), sep='\t', index=False, header=False)

In [0]:
kf = StratifiedKFold(n_splits=5)
idx = [x for x in kf.split(train, train.wt_freq)]
test_pred = np.zeros((len(test), 48))
train_copy = train.copy()
for i, (tr_idx, val_idx) in enumerate(idx[0:1], 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train_copy.loc[tr_idx], train_copy.loc[val_idx]
    convert_data(tr)
    convert_data(val, 'dev')
    convert_data(test, 'test')
    os.system(
        'python /content/ALBERT-TF2.0/create_finetuning_data.py \
        --input_data_dir=/content/input \
        --spm_model_file=/content/drive/My\ Drive/albert_model/base/vocab/30k-clean.model \
        --train_data_output_path=/content/input/train.tf_record \
        --eval_data_output_path=/content/input/eval.tf_record \
        --meta_data_file_path=/content/input/meta_data \
        --fine_tuning_task_type=classification \
        --max_seq_length=142 \
        --classification_task_name=COLA'
        )

    # os.system(
       !python /content/ALBERT-TF2.0/run_classifer.py \
        --train_data_path=/content/input/train.tf_record \
        --eval_data_path=/content/input/eval.tf_record \
        --predict_data_path=/content/input/predict.tf_record \
        --input_meta_data_path=/content/input/meta_data \
        --albert_config_file=/content/drive/My\ Drive/albert_model/base/config.json \
        --task_name=CoLA \
        --spm_model_file=/content/drive/My\ Drive/albert_model/base/vocab/30k-clean.model \
        --output_dir=/content/output \
        --init_checkpoint=/content/drive/My\ Drive/albert_model/base/tf2_model.h5 \
        --do_train \
        --do_eval \
        --do_predict \
        --custom_training_loop=True \
        --train_batch_size=16 \
        --eval_batch_size=8 \
        --learning_rate=1e-4 \
        --max_seq_length=142 \
        --num_train_epochs=2
        # )





    

FATAL Flags parsing error: Unknown command line flag 'predict_data_path'. Did you mean: train_data_path ?
Pass --helpshort or --helpfull to see help on flags.


## ALbert-Keras

In [13]:
!pip install git+https://github.com/TinkerMob/keras_albert_model.git
!git clone https://github.com/acmilannesta/Bert-embedding
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip -A


Archive:  uncased_L-12_H-768_A-12.zip
replace uncased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace uncased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
import json
import numpy as np
import pandas as pd
from random import choice
import re, os, gc
import codecs
from keras.layers import *
from keras.losses import sparse_categorical_crossentropy
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam, SGD
from keras.callbacks import Callback, LearningRateScheduler
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from tqdm import tqdm
from functools import partial
from keras_bert import AdamWarmup, calc_train_steps, Tokenizer
from keras_albert_model import build_albert, load_brightmart_albert_zh_checkpoint
import warnings
warnings.filterwarnings("ignore")

In [0]:
train = pd.read_csv('Bert-embedding/CDC/train.csv')
test = pd.read_csv('Bert-embedding/CDC/test.csv')
# Event weight
wt = pd.DataFrame(train.event.value_counts()/len(train)).rename(columns={'event':'weight'})
wt['event'] = wt.index
train = train.merge(wt, how='left', on='event')
# Reassign eventcode
train['event_idx'] = train.event.map({y:x for x, y in enumerate(np.sort(train.event.unique()))})
# Assign weight freqency
train['wt_freq'] = np.where(train.weight<0.01, 1, np.where(train.weight<0.05, 2, 3))

## Parameter setting

In [0]:
MAXLEN = 142 #@param {type:'slider', min:50, max:300, step:1}
BATCH_SIZE = 16 #@param {type:'slider', min:8, max:32, step:8}
NUM_EPOCHS = 3
NUM_CLASSES = 48
LR = 5e-5
config_path = 'uncased_L-12_H-768_A-12/bert_config.json' #@param ['uncased_L-12_H-768_A-12/bert_config.json', 'wwm_uncased_L-24_H-1024_A-16/bert_config.json', '/content/drive/My Drive/biobert_pretrain_output_all_notes_150000/bert_config.json']
checkpoint_path = 'uncased_L-12_H-768_A-12/bert_model.ckpt' #@param ['uncased_L-12_H-768_A-12/bert_model.ckpt', 'wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt', '/content/drive/My Drive/biobert_pretrain_output_all_notes_150000/model.ckpt']
dict_path = 'uncased_L-12_H-768_A-12/vocab.txt' #@param ['uncased_L-12_H-768_A-12/vocab.txt', 'wwm_uncased_L-24_H-1024_A-16/vocab.txt', '/content/drive/My Drive/biobert_pretrain_output_all_notes_150000/vocab.txt']

## Tokenize train and validation set

In [0]:
token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict)

In [0]:
def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df.loc[i, 'text'])
        indices.append(ids)
    aux = data_df[['age', 'sex']].apply(lambda x: (x - min(x)) / (max(x)-min(x)))
    if branch=='training':
        targets = data_df['event_idx'] 
        return indices, np.array(targets), np.array(aux)
    else:
        return indices, np.array(aux)


## Data Generator

In [0]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X])

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])           
                X2 = np.zeros_like(X1)
                if self.branch == 'test':
                    aux = np.array([x[1] for x in d])
                    yield [X1, X2, aux]
                else:
                    Y = np.array([x[1] for x in d])
                    aux = np.array([x[2] for x in d])
                    yield [X1, X2, aux], Y

##Model Assemble

In [12]:
def model_build(len_train):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global LR

    albert_model = load_brightmart_albert_zh_checkpoint('uncased_L-12_H-768_A-12')
    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))
    aux_in = Input(shape=(2, ))

    inputs = albert_model([x1_in, x2_in])
    dense = Lambda(lambda x: x[:, 0])(inputs)
    dense = concatenate([dense, aux_in])
    outputs = Dense(NUM_CLASSES, activation='softmax')(dense)
    model = Model([x1_in, x2_in, aux_in], outputs)

    decay_steps, warmup_steps = calc_train_steps(
        len_train, #tr_pseudo_x
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
    )

    # def custom_loss(y_true, y_pred, w):
    #     return sparse_categorical_crossentropy(y_true, y_pred) * w

    model.compile(
        loss = 'sparse_categorical_crossentropy',
        # loss=partial(custom_loss, w = w_in),
        # optimizer=Adam(1e-5),
        optimizer=AdamWarmup(
            decay_steps=decay_steps, 
            warmup_steps=warmup_steps, 
            lr=LR, 
            # min_lr=1e-6,
            ),
        metrics = ['sparse_categorical_accuracy']
    )
    del albert_model
    gc.collect()
    return model

KeyError: ignored

In [0]:
test_indices, test_aux = convert_data(test, branch='test')
kf = StratifiedKFold(n_splits=5, random_state=0)
pred = np.zeros((len(test), NUM_CLASSES))
idx = [x for x in kf.split(train, train.wt_freq)]


for i, (tr_idx, val_idx) in enumerate(idx, 1):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    tr_x, tr_y, tr_aux = convert_data(tr)
    val_x, val_y, val_aux = convert_data(val)
    model = model_build(len_train=len(tr_x))
    train_D = data_generator(list(zip(tr_x, tr_y, tr_aux)))
    valid_D = data_generator(list(zip(val_x, val_y, val_aux)), branch='valid')
    # ival = IntervalEvaluation(validation_data=valid_D, label=val_y, interval=len(train_D)*NUM_EPOCHS)
    model.fit_generator(    
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=NUM_EPOCHS,
        # callbacks = [ival]
    )
    oof_pred = model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    train.loc[val_idx, 'oof_pred'] = np.argmax(oof_pred, 1)
    print('oof - {:} f1_score - {:.4f}'.format(i, f1_score(val_y, np.argmax(oof_pred, 1), average='weighted')))
    test_D = data_generator(list(zip(test_indices, test_aux)), branch='test')
    pred += model.predict_generator(test_D.__iter__(), len(test_D), verbose=1) / kf.get_n_splits()
    del model
    gc.collect()
    train.to_csv('drive/My Drive/CDC Model/oof/train_albert_oof.csv', index=False)
    np.save('drive/My Drive/CDC Model/oof/test_albert_oof.npy', pred)


In [14]:
!git clone https://github.com/acmilannesta/eda_nlp

Cloning into 'eda_nlp'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 379 (delta 18), reused 0 (delta 0), pack-reused 349
Receiving objects: 100% (379/379), 20.41 MiB | 17.13 MiB/s, done.
Resolving deltas: 100% (181/181), done.


In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [23]:
train[['event', 'text']].to_csv('input.tsv', sep='\t', index=False, header=False)
len(train)*2

307912

In [22]:
!python eda_nlp/code/augment.py --input=input.tsv --output=output.tsv --num_aug=1 --alpha=0.05

generated augmented sentences with eda for input.tsv to output.tsv with num_aug=1


In [28]:
output = pd.read_csv('output.tsv', sep='\t', header=None)
output.head(5)

,0,1
0,62,yom with contusion to face after spell strikin...
1,62,yom with contusion to face after striking it w...
2,42,a yom fell on arm while working had slipped on...
3,42,a yom fell on arm while working had slipped on...
4,26,yom with cervical strain back pain s p taxi dr...
